In [1]:
# Setup library
## install -r requirements.txt
from __future__ import absolute_import, division, print_function, unicode_literals
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import tensorflow as tf
keras = tf.keras

In [2]:
# Prepare the base model from the pre-trained model MobileNet V2 by Google Inc.
## https://arxiv.org/abs/1801.04381
IMG_SHAPE = (224, 224) # default shape: [96, 128, 160, 192, 224]

## only feature extraction model
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE + (3,),
                                               include_top=False,
                                               weights='imagenet')

In [3]:
# prepare dataset
dataset_root = os.path.abspath(os.path.expanduser('./data/'))
print(f'Dataset root: {dataset_root}')

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                  validation_split=0.1)
train_data = image_generator.flow_from_directory(dataset_root, target_size=IMG_SHAPE,
                                                 subset='training')
validation_data = image_generator.flow_from_directory(dataset_root, target_size=IMG_SHAPE,
                                                 subset='validation')

for image_batch, label_batch in validation_data:
    print(f'Image batch shape: {image_batch.shape}')
    print(f'Label batch shape: {label_batch.shape}')
    feature_batch = base_model(image_batch)
    print(f'Feature batch shape: {feature_batch.shape}')
    break

Dataset root: /home/harny/Github/cell-classification/data
Found 842 images belonging to 3 classes.
Found 92 images belonging to 3 classes.
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 3)
Feature batch shape: (32, 7, 7, 1280)


In [4]:
# Freeze the convolutional base model
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
# The classification layer using feature vector
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(f'Feature vector shape: {feature_batch_average.shape}')

prediction_layer = keras.layers.Dense(len(train_data.class_indices), activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(f'Prediction result shape: {prediction_batch.shape}')

Feature vector shape: (32, 1280)
Prediction result shape: (32, 3)


In [6]:
# Create the model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

# Compile model
base_learning_rate = 0.0001
# base_learning_rate = 0.001 # default
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate), # Optimizer
              loss=tf.keras.losses.CategoricalCrossentropy(), # Loss
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [7]:
# before train, we check the initial accuracy.
validation_steps = np.ceil(validation_data.samples/validation_data.batch_size)

loss0, accuracy0 = model.evaluate(validation_data, steps=validation_steps)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

  ...
    to  
  ['...']
3/3 [==============================] - 4s 1s/step - loss: 1.4165 - accuracy: 0.3043
initial loss: 1.42
initial accuracy: 0.30


In [8]:
# model fit
initial_epochs = 100
history = model.fit(train_data,
                    epochs=initial_epochs,
                    validation_data=validation_data)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 27 steps, validate for 3 steps
Epoch 1/100
27/27 [==============================] - 36s 1s/step - loss: 1.1050 - accuracy: 0.4608 - val_loss: 1.2895 - val_accuracy: 0.3261
Epoch 2/100
27/27 [==============================] - 33s 1s/step - loss: 0.9949 - accuracy: 0.5143 - val_loss: 1.2951 - val_accuracy: 0.3696
Epoch 3/100
27/27 [==============================] - 33s 1s/step - loss: 0.9023 - accuracy: 0.5855 - val_loss: 1.2852 - val_accuracy: 0.4022
Epoch 4/100
27/27 [==============================] - 33s 1s/step - loss: 0.8294 - accuracy: 0.6366 - val_loss: 1.2939 - val_accuracy: 0.4130
Epoch 5/100
27/27 [==============================] - 33s 1s/step - loss: 0.7667 - accuracy: 0.6900 - val_loss: 1.2888 - val_accuracy: 0.4239
Epoch 6/100
27/27 [==============================] - 33s 1s/step - loss: 0.7157 - accuracy: 0.7185 - val_loss: 1.3016 - val_accuracy: 0.4348
Epoch 7/100
27/27 [==============================] - 33s 1s/ste

In [9]:
# Draw Learning curves chart
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Finetune preperation
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

# Frozen after some point
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  155


In [11]:
# Recompile the model for finetune
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate/10), # Optimizer
              loss=tf.keras.losses.CategoricalCrossentropy(), # Loss
              metrics=['accuracy'])
model.summary()
len(model.trainable_variables)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 1,866,435
Non-trainable params: 395,392
_________________________________________________________________


58

In [12]:
# Finetune fitting
fine_tune_epochs = 100
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_data,
                         epochs=total_epochs,
                         initial_epoch = history.epoch[-1],
                         validation_data=validation_data)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 27 steps, validate for 3 steps
Epoch 100/200
27/27 [==============================] - 40s 1s/step - loss: 0.1307 - accuracy: 0.9774 - val_loss: 1.6524 - val_accuracy: 0.4891
Epoch 101/200
27/27 [==============================] - 37s 1s/step - loss: 0.0805 - accuracy: 0.9952 - val_loss: 1.6755 - val_accuracy: 0.5326
Epoch 102/200
27/27 [==============================] - 37s 1s/step - loss: 0.0578 - accuracy: 1.0000 - val_loss: 1.6660 - val_accuracy: 0.5326
Epoch 103/200
27/27 [==============================] - 37s 1s/step - loss: 0.0427 - accuracy: 1.0000 - val_loss: 1.6817 - val_accuracy: 0.5435
Epoch 104/200
27/27 [==============================] - 38s 1s/step - loss: 0.0336 - accuracy: 1.0000 - val_loss: 1.7183 - val_accuracy: 0.5435
Epoch 105/200
27/27 [==============================] - 37s 1s/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 1.7321 - val_accuracy: 0.5435
Epoch 106/200
27/27 [==============================

In [13]:
# prepare learning curves data with finetune
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [14]:
# Draw Learning curves chart include finetune result
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …